In [126]:
from pathlib import Path
import kagglehub
import pandas as pd
import pandera.pandas as pa
import warnings

import numpy as np

In [2]:
path = kagglehub.dataset_download("paultimothymooney/stock-market-data/")

print("Path to dataset files:", path)

100%|█████████████████████████████████████| 1.03G/1.03G [06:23<00:00, 2.89MB/s]

Extracting files...


Path to dataset files: C:\Users\mateu\.cache\kagglehub\datasets\paultimothymooney\stock-market-data\versions\74


In [42]:
PATH_RAW = Path("raw")

In [43]:
!ls C:\Users\mateu\.cache\kagglehub\datasets\paultimothymooney\stock-market-data\versions\74\stock_market_data\sp500\csv\GOOG.csv

C:\Users\mateu\.cache\kagglehub\datasets\paultimothymooney\stock-market-data\versions\74\stock_market_data\sp500\csv\GOOG.csv


In [44]:
sp500_path = Path(path + "/stock_market_data/sp500/csv/GOOG.csv")

In [45]:
df_goog = pd.read_csv(sp500_path, parse_dates=["Date"], index_col="Date", usecols=["Date","Adjusted Close"], dayfirst=True)

In [46]:
df_goog

,Adjusted Close
Date,
2004-08-19,2.499133
2004-08-20,2.697639
2004-08-23,2.724787
2004-08-24,2.611960
2004-08-25,2.640104
...,...
2022-12-06,97.309998
2022-12-07,95.150002
2022-12-08,93.949997


In [47]:
df_goog["Adjusted Close"].pct_change()

Date
2004-08-19         NaN
2004-08-20    0.079430
2004-08-23    0.010064
2004-08-24   -0.041408
2004-08-25    0.010775
                ...   
2022-12-06   -0.025633
2022-12-07   -0.022197
2022-12-08   -0.012612
2022-12-09   -0.009367
2022-12-12   -0.006393
Name: Adjusted Close, Length: 4612, dtype: float64

In [48]:
df_goog["LogReturn"] = np.log(df_goog["Adjusted Close"] / df_goog["Adjusted Close"].shift(1))

In [49]:
df_goog

,Adjusted Close,LogReturn
Date,,
2004-08-19,2.499133,NaN
2004-08-20,2.697639,0.076433
2004-08-23,2.724787,0.010013
2004-08-24,2.611960,-0.042289
2004-08-25,2.640104,0.010717
...,...,...
2022-12-06,97.309998,-0.025968
2022-12-07,95.150002,-0.022447
2022-12-08,93.949997,-0.012692


In [50]:
df_goog = df_goog.drop(columns=["Adjusted Close"])

In [51]:
df_goog = df_goog.dropna()

In [52]:
df_goog.head()

,LogReturn
Date,
2004-08-20,0.076433
2004-08-23,0.010013
2004-08-24,-0.042289
2004-08-25,0.010717
2004-08-26,0.017859


In [55]:
df_goog.to_parquet(PATH_RAW/ "googl_returns.parquet", index=True)

In [21]:
df_goog = pd.read_parquet("raw/googl_returns.parquet")

In [22]:
df_goog.shape

(4611, 1)

In [26]:
df_goog.loc['2021'].to_parquet("raw/googl_returns_2021.parquet", index=True)

In [171]:
df_goog = pd.read_parquet("raw/googl_returns_2021.parquet")

In [172]:
df_goog

,LogReturn
Date,
2021-01-04,-0.013586
2021-01-05,0.007310
2021-01-06,-0.003239
2021-01-07,0.029504
2021-01-08,0.011106
...,...
2021-12-27,0.006243
2021-12-28,-0.010974
2021-12-29,0.000386


In [173]:
df_sp500 = pd.read_csv("INDEX_US_S&P_US_SPX_2021.csv", parse_dates=["Date"], index_col="Date", usecols=["Date", "Close"], thousands=',').sort_index()

In [174]:
df_sp500.shape

(252, 1)

In [175]:
df_sp500

,Close
Date,
2021-01-04,3700.65
2021-01-05,3726.86
2021-01-06,3748.14
2021-01-07,3803.79
2021-01-08,3824.68
...,...
2021-12-27,4791.19
2021-12-28,4786.35
2021-12-29,4793.06


In [176]:
df_sp500.dtypes

Close    float64
dtype: object

In [177]:
df_sp500["LogReturn"] = np.log(df_sp500["Close"] / df_sp500["Close"].shift(1))

In [178]:
df_sp500 = df_sp500.drop(columns=["Close"])

In [179]:
df_sp500 = df_sp500.dropna()

In [180]:
df_sp500

,LogReturn
Date,
2021-01-05,0.007058
2021-01-06,0.005694
2021-01-07,0.014738
2021-01-08,0.005477
2021-01-11,-0.006576
...,...
2021-12-27,0.013744
2021-12-28,-0.001011
2021-12-29,0.001401


In [183]:
#pandera
schema_log_returns = pa.DataFrameSchema(
    {
        "LogReturn": pa.Column(
            float,
            checks=[
                pa.Check.greater_than(-1),
                pa.Check.less_than(1)
            ],
            nullable=False,
        )
    },
    index=pa.Index(
        pa.DateTime,
        name="Date",
        checks=[
            pa.Check.in_range('2021-01-01', '2021-12-31'),
            # pa.Check(lambda idx: idx.is_monotonic_incressing, element_wise=False)
        ]
    ),
    strict=True, #nie moze miec wiecej kolumn niz tu mamy zdefiniownaych
    coerce=True
)

'0.26.1'

In [184]:
schema_log_returns.validate(df_sp500)

,LogReturn
Date,
2021-01-05,0.007058
2021-01-06,0.005694
2021-01-07,0.014738
2021-01-08,0.005477
2021-01-11,-0.006576
...,...
2021-12-27,0.013744
2021-12-28,-0.001011
2021-12-29,0.001401


In [185]:
schema_log_returns.validate(df_goog)

,LogReturn
Date,
2021-01-04,-0.013586
2021-01-05,0.007310
2021-01-06,-0.003239
2021-01-07,0.029504
2021-01-08,0.011106
...,...
2021-12-27,0.006243
2021-12-28,-0.010974
2021-12-29,0.000386


In [186]:
df_sp500.to_parquet("raw/sp500_returns_2021.parquet", index=True)